In [1]:
import torch
import torch.nn as nn
import transformers
from math import sqrt
from transformers import LlamaConfig, LlamaModel, LlamaTokenizer
from transformers import GPT2Config, GPT2Tokenizer, GPT2Model, AutoTokenizer
from layers.Embed import PatchEmbeddingTimeLLM
from layers.Transformer_EncDec import Decoder, DecoderLayer, Encoder, EncoderLayer, ConvLayer
from layers.SelfAttention_Family import FullAttention, AttentionLayer
from layers.StandardNorm import Normalize
from layers.fusion import GatedFusion
from utils import prompt_helper
import torch.nn.functional as F

KeyboardInterrupt: 

In [3]:
tokenizer = LlamaTokenizer.from_pretrained(
            "/data/LLM/models--meta-llama--Llama-2-13b-chat-hf/snapshots/a2cb7a712bb6e5e736ca7f8cd98167f81a0b5bd8",
            # 'huggyllama/llama-7b',
            trust_remote_code=True,
            local_files_only=True
        )
if tokenizer.eos_token:
    tokenizer.pad_token = tokenizer.eos_token
else:
    pad_token = '[PAD]'
    tokenizer.add_special_tokens({'pad_token': pad_token})
    tokenizer.pad_token = pad_token

In [4]:
inputs = tokenizer(["Today is 23 412"], return_tensors="pt")
inputs

{'input_ids': tensor([[    1, 20628,   338, 29871, 29906, 29941, 29871, 29946, 29896, 29906]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
# Load model directly
from huggingface_hub import snapshot_download

snapshot_download(repo_id="meta-llama/Llama-3.1-70B-Instruct", cache_dir='/data/LLMs/')

Fetching 50 files:   0%|          | 0/50 [00:00<?, ?it/s]

model-00001-of-00030.safetensors:   0%|          | 0.00/4.58G [00:00<?, ?B/s]

model-00003-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00008-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

...

model-00015-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00016-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00017-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00018-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00019-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

In [1]:
from models.BatteryLifeLLM import BatteryLifeLLM
from BatteryLifeLLMUtils.configuration_BatteryLifeLLM import BatteryElectrochemicalConfig
battery_config = BatteryElectrochemicalConfig()
a = BatteryLifeLLM()
print(a.config_class)

2024-05-28 15:28:10.406811: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 15:28:11.175826: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ValueError: Parameter config in `BatteryLifeLLM(config)` should be an instance of class `PretrainedConfig`. To create a model from a pretrained model use `model = BatteryLifeLLM.from_pretrained(PRETRAINED_MODEL_NAME)`

In [3]:
import json
data = json.load(open('./dataset/MIT_LFP_Batch9_cell_data.json'))
len(data)

import os
a = os.listdir('./dataset/MIT_dataset')
len(a)

124

统计一下MIT数据集有多少种不同的prompt

In [3]:
from Prompts import HUST_protocol_prompt, MIT_all_protocol_prompt
prompts = HUST_protocol_prompt.HUST_protocol_prompt.Protocols_prompts
prompt_keys = {}
for key, value in prompts.items():
    prompt_keys[value] = prompt_keys.get(value, []) + [key]

for key, value in prompt_keys.items():
    if len(value) > 1:
        print(key, value)
print(len(prompt_keys))

77
